In [95]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from datetime import datetime, timedelta


edge_driver_path = 'C:/Drivers/msedgedriver.exe'  


options = Options()
options.add_argument('--start-maximized')  


def set_date(driver, date_input_id, date):
    """Set the date in the date picker by inputting a date."""
    date_input = driver.find_element(By.ID, date_input_id)
    date_input.send_keys(Keys.CONTROL + "a")
    date_input.send_keys(Keys.BACKSPACE)
    formatted_date = date.strftime("%m.%d.%Y")  
    date_input.send_keys(formatted_date)
    date_input.send_keys(Keys.RETURN)

def scrape_ports(url, driver_path, options, start_date, end_date):
    """Scrapes port data from the provided URL for a range of dates."""
    driver = webdriver.Edge(service=Service(executable_path=driver_path), options=options)
    driver.get(url)
    time.sleep(5)  

    all_data = []  

    try:
        iframe = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, "iframe"))
        )
        driver.switch_to.frame(iframe)

        current_date = start_date
        while current_date <= end_date:
            set_date(driver, "mat-input-0", current_date)  
            time.sleep(3)  

            html_content = driver.page_source
            soup = BeautifulSoup(html_content, 'html.parser')

            ports = soup.find_all('li', class_='elevator-list-item')

            for port in ports:
                port_title = port.find('h3', class_='elevator-inform__title').get_text(strip=True)
                
                prices_section = port.find_all('li', class_='list-item')
                for price_item in prices_section:
                    crop_name_span = price_item.find_all('span', recursive=False)[0] 
                    crop_name = crop_name_span.get_text(strip=True) if crop_name_span else ""

                    price_span = price_item.find('span', class_='pull-right color-bl')
                    if not price_span:
                        price_span = price_item.find('span', class_='ng-star-inserted')

                    price = price_span.get_text(strip=True) if price_span else "Price not found"

                    all_data.append({
                        'Port Title': port_title,
                        'Crop': crop_name,
                        'Price': price,
                        'Date': current_date.strftime("%m.%d.%Y")  
                    })

            current_date += timedelta(days=1)

        df = pd.DataFrame(all_data)
        return df  

    finally:
        driver.quit()


def scrape_elevators(soup, date_text):
    """ Scrapes elevator data from the current city page. """
    all_data = [] 

    elevators = soup.find_all('li', class_='elevator-list-item')

    for elevator in elevators:
        title = elevator.find('h3', class_='elevator-title')
        title = title.get_text(strip=True) if title else "Title not found"

        address = elevator.find('span', class_='elevator-address__name')
        address = address.get_text(strip=True) if address else "Address not found"
        """
        if address:
            cleaned_address = clean_address(address.get_text(strip=True))
        else:
            cleaned_address = "Address not found"
        """
        director_name = elevator.find('span', class_='elevator-direction_name')
        director_name = director_name.get_text(strip=True) if director_name else ''
        
        director_phone_element = elevator.find('a', href=True)
        director_phone = director_phone_element.get_text(strip=True) if director_phone_element else ''
        
        # Scrape manager's information
        manager_info = elevator.find_all('span', class_='elevator-direction_name')[1] if len(elevator.find_all('span', class_='elevator-direction_name')) > 1 else None
        manager_name = manager_info.get_text(strip=True) if manager_info else ''
        
        manager_phone_element = elevator.find_all('a', href=True)[1] if len(elevator.find_all('a', href=True)) > 1 else None
        manager_phone = manager_phone_element.get_text(strip=True) if manager_phone_element else ''

        prices_section = elevator.find_all('li', class_='list-item')
        for price_item in prices_section:
            crop_name = price_item.find('span', class_='culture-name').get_text(strip=True)
            price = price_item.find('span', class_='ng-star-inserted').get_text(strip=True)

            all_data.append({
                'Elevator Title': title,
                'Address': address,
                'Crop': crop_name,
                'Director Name': director_name,
                'Director Phone': director_phone,
                'Manager Name': manager_name,
                'Manager Phone': manager_phone,    
                'Price': price,
                'Date' : date_text
            })

    return all_data
"""
obl_names = [
    "Сумська", "Вінницька", "Харківська", 
    "Хмельницька", "Рівненська", "Миколаївська"
]

def clean_address(address):
     Function to extract the Oblast name from the address. 
    for obl_name in obl_names:
        if obl_name in address:
            return obl_name  
    return "Oblast not found"  
"""
def scrape_all_cities(url, driver_path, options, start_date, end_date):
    """ Scrapes all cities and their elevator data using iframes and loops through the date range. """
    driver = webdriver.Edge(service=Service(executable_path=driver_path), options=options)
    
    driver.get(url)
    time.sleep(5)

    all_data = [] 
    
    try:
        iframe = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, "iframe"))
        )
        driver.switch_to.frame(iframe)

        cities_list = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "ul.cities > li"))
        )

        current_date = start_date
        while current_date <= end_date:
            set_date(driver, "mat-input-0", current_date)  
            time.sleep(3)  

            for city in cities_list:
                city.click()
                time.sleep(2)  

                html_content = driver.page_source
                soup = BeautifulSoup(html_content, 'html.parser')

                date_element = soup.find('span', class_='last-update')
                date_text = date_element.find_all('span')[-1].get_text(strip=True) if date_element else "Date not found"

                city_data = scrape_elevators(soup, date_text)
                all_data.extend(city_data) 

            current_date += timedelta(days=1)

        df = pd.DataFrame(all_data)
        return df 

    finally:
        driver.quit()

def scrape_ports_and_elevators_with_date_range(start_date_str, end_date_str):
    start_date = datetime.strptime(start_date_str, "%m.%d.%Y")  
    end_date = datetime.strptime(end_date_str, "%m.%d.%Y") 

    ports_url = 'https://agroprosperis.com/price-in-ports.html'
    ports_df = scrape_ports(ports_url, edge_driver_path, options, start_date, end_date)
    print("Port Data Scraped:")
    print(ports_df)

    elevators_url = 'https://agroprosperis.com/grain-marketing.html'
    elevators_df = scrape_all_cities(elevators_url, edge_driver_path, options, start_date, end_date)
    print("Elevator Data Scraped:")
    print(elevators_df)

    combined_df = pd.concat([ports_df, elevators_df], ignore_index=True)
    return combined_df


start_date_input = "01.01.2020"  
end_date_input = "09.17.2024"  

df = scrape_ports_and_elevators_with_date_range(start_date_input, end_date_input)

print(df)


Port Data Scraped:
               Port Title                      Crop              Price  \
0      Тіс-Міндобрива ТОВ  Соя(37.5% білок на суху)  8,450 UAH/355 USD   
1           Тіс-Зерно ТОВ            Пшениця 2 клас  5,535 UAH/197 USD   
2           Тіс-Зерно ТОВ            Пшениця 3 клас  5,535 UAH/197 USD   
3           Тіс-Зерно ТОВ            Пшениця 4 клас  5,420 UAH/193 USD   
4           Тіс-Зерно ТОВ          Кукурудза 3 клас  4,635 UAH/165 USD   
...                   ...                       ...                ...   
12181       Тіс-Зерно ТОВ          Кукурудза 3 клас  8,500 UAH/180 USD   
12182      М.В. Карго ТОВ             Ячмінь 3 клас  7,900 UAH/168 USD   
12183  Тіс-Міндобрива ТОВ            Пшениця 2 клас  9,200 UAH/196 USD   
12184  Тіс-Міндобрива ТОВ            Пшениця 3 клас  9,100 UAH/194 USD   
12185  Тіс-Міндобрива ТОВ            Пшениця 4 клас  8,700 UAH/184 USD   

             Date  
0      01.01.2020  
1      01.01.2020  
2      01.01.2020  
3      01.01

In [96]:
df

,Port Title,Crop,Price,Date,Elevator Title,Address,Director Name,Director Phone,Manager Name,Manager Phone
0,Тіс-Міндобрива ТОВ,Соя(37.5% білок на суху),"8,450 UAH/355 USD",01.01.2020,NaN,NaN,NaN,NaN,NaN,NaN
1,Тіс-Зерно ТОВ,Пшениця 2 клас,"5,535 UAH/197 USD",01.01.2020,NaN,NaN,NaN,NaN,NaN,NaN
2,Тіс-Зерно ТОВ,Пшениця 3 клас,"5,535 UAH/197 USD",01.01.2020,NaN,NaN,NaN,NaN,NaN,NaN
3,Тіс-Зерно ТОВ,Пшениця 4 клас,"5,420 UAH/193 USD",01.01.2020,NaN,NaN,NaN,NaN,NaN,NaN
4,Тіс-Зерно ТОВ,Кукурудза 3 клас,"4,635 UAH/165 USD",01.01.2020,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
122973,NaN,Соняшник некласний,21000 грн.,17.09.2024,Радивилівський елеватор ТОВ,"35500, Рівненська обл., Дубенський р-н, м. Рад...",Порчинський Сергій Іванович,+38 (067) 380-34-33,Мельник Анатолій Іванович,+38 (067) 645-16-82
122974,NaN,Пшениця 2 клас,8300 грн.,17.09.2024,ДПЗКУ ПАТ (філія Новополтавський елеватор),"55645, Миколаївська, обл., Баштанський, р-н",,,,
122975,NaN,Пшениця 3 клас,8200 грн.,17.09.2024,ДПЗКУ ПАТ (філія Новополтавський елеватор),"55645, Миколаївська, обл., Баштанський, р-н",,,,
122976,NaN,Пшениця 4 клас,7400 грн.,17.09.2024,ДПЗКУ ПАТ (філія Новополтавський елеватор),"55645, Миколаївська, обл., Баштанський, р-н",,,,


In [97]:
df_copy = df.copy()

In [98]:
df_copy['Price'] = df_copy['Price'].str.split().str[0]

df_copy['Price'] = df_copy['Price'].str.replace(r'[^\d.]', '', regex=True).astype(int)

df_copy


,Port Title,Crop,Price,Date,Elevator Title,Address,Director Name,Director Phone,Manager Name,Manager Phone
0,Тіс-Міндобрива ТОВ,Соя(37.5% білок на суху),8450,01.01.2020,NaN,NaN,NaN,NaN,NaN,NaN
1,Тіс-Зерно ТОВ,Пшениця 2 клас,5535,01.01.2020,NaN,NaN,NaN,NaN,NaN,NaN
2,Тіс-Зерно ТОВ,Пшениця 3 клас,5535,01.01.2020,NaN,NaN,NaN,NaN,NaN,NaN
3,Тіс-Зерно ТОВ,Пшениця 4 клас,5420,01.01.2020,NaN,NaN,NaN,NaN,NaN,NaN
4,Тіс-Зерно ТОВ,Кукурудза 3 клас,4635,01.01.2020,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
122973,NaN,Соняшник некласний,21000,17.09.2024,Радивилівський елеватор ТОВ,"35500, Рівненська обл., Дубенський р-н, м. Рад...",Порчинський Сергій Іванович,+38 (067) 380-34-33,Мельник Анатолій Іванович,+38 (067) 645-16-82
122974,NaN,Пшениця 2 клас,8300,17.09.2024,ДПЗКУ ПАТ (філія Новополтавський елеватор),"55645, Миколаївська, обл., Баштанський, р-н",,,,
122975,NaN,Пшениця 3 клас,8200,17.09.2024,ДПЗКУ ПАТ (філія Новополтавський елеватор),"55645, Миколаївська, обл., Баштанський, р-н",,,,
122976,NaN,Пшениця 4 клас,7400,17.09.2024,ДПЗКУ ПАТ (філія Новополтавський елеватор),"55645, Миколаївська, обл., Баштанський, р-н",,,,


In [99]:
import numpy as np

df_copy['Terminal type'] = np.where(df_copy['Port Title'].notna(), 'Port', 
                                    np.where(df_copy['Elevator Title'].notna(), 'Elevator', np.nan))

df_copy

,Port Title,Crop,Price,Date,Elevator Title,Address,Director Name,Director Phone,Manager Name,Manager Phone,Terminal type
0,Тіс-Міндобрива ТОВ,Соя(37.5% білок на суху),8450,01.01.2020,NaN,NaN,NaN,NaN,NaN,NaN,Port
1,Тіс-Зерно ТОВ,Пшениця 2 клас,5535,01.01.2020,NaN,NaN,NaN,NaN,NaN,NaN,Port
2,Тіс-Зерно ТОВ,Пшениця 3 клас,5535,01.01.2020,NaN,NaN,NaN,NaN,NaN,NaN,Port
3,Тіс-Зерно ТОВ,Пшениця 4 клас,5420,01.01.2020,NaN,NaN,NaN,NaN,NaN,NaN,Port
4,Тіс-Зерно ТОВ,Кукурудза 3 клас,4635,01.01.2020,NaN,NaN,NaN,NaN,NaN,NaN,Port
...,...,...,...,...,...,...,...,...,...,...,...
122973,NaN,Соняшник некласний,21000,17.09.2024,Радивилівський елеватор ТОВ,"35500, Рівненська обл., Дубенський р-н, м. Рад...",Порчинський Сергій Іванович,+38 (067) 380-34-33,Мельник Анатолій Іванович,+38 (067) 645-16-82,Elevator
122974,NaN,Пшениця 2 клас,8300,17.09.2024,ДПЗКУ ПАТ (філія Новополтавський елеватор),"55645, Миколаївська, обл., Баштанський, р-н",,,,,Elevator
122975,NaN,Пшениця 3 клас,8200,17.09.2024,ДПЗКУ ПАТ (філія Новополтавський елеватор),"55645, Миколаївська, обл., Баштанський, р-н",,,,,Elevator
122976,NaN,Пшениця 4 клас,7400,17.09.2024,ДПЗКУ ПАТ (філія Новополтавський елеватор),"55645, Миколаївська, обл., Баштанський, р-н",,,,,Elevator


In [100]:
df_copy['Port Title'] = df_copy['Port Title'].fillna(df_copy['Elevator Title'])

df_copy


,Port Title,Crop,Price,Date,Elevator Title,Address,Director Name,Director Phone,Manager Name,Manager Phone,Terminal type
0,Тіс-Міндобрива ТОВ,Соя(37.5% білок на суху),8450,01.01.2020,NaN,NaN,NaN,NaN,NaN,NaN,Port
1,Тіс-Зерно ТОВ,Пшениця 2 клас,5535,01.01.2020,NaN,NaN,NaN,NaN,NaN,NaN,Port
2,Тіс-Зерно ТОВ,Пшениця 3 клас,5535,01.01.2020,NaN,NaN,NaN,NaN,NaN,NaN,Port
3,Тіс-Зерно ТОВ,Пшениця 4 клас,5420,01.01.2020,NaN,NaN,NaN,NaN,NaN,NaN,Port
4,Тіс-Зерно ТОВ,Кукурудза 3 клас,4635,01.01.2020,NaN,NaN,NaN,NaN,NaN,NaN,Port
...,...,...,...,...,...,...,...,...,...,...,...
122973,Радивилівський елеватор ТОВ,Соняшник некласний,21000,17.09.2024,Радивилівський елеватор ТОВ,"35500, Рівненська обл., Дубенський р-н, м. Рад...",Порчинський Сергій Іванович,+38 (067) 380-34-33,Мельник Анатолій Іванович,+38 (067) 645-16-82,Elevator
122974,ДПЗКУ ПАТ (філія Новополтавський елеватор),Пшениця 2 клас,8300,17.09.2024,ДПЗКУ ПАТ (філія Новополтавський елеватор),"55645, Миколаївська, обл., Баштанський, р-н",,,,,Elevator
122975,ДПЗКУ ПАТ (філія Новополтавський елеватор),Пшениця 3 клас,8200,17.09.2024,ДПЗКУ ПАТ (філія Новополтавський елеватор),"55645, Миколаївська, обл., Баштанський, р-н",,,,,Elevator
122976,ДПЗКУ ПАТ (філія Новополтавський елеватор),Пшениця 4 клас,7400,17.09.2024,ДПЗКУ ПАТ (філія Новополтавський елеватор),"55645, Миколаївська, обл., Баштанський, р-н",,,,,Elevator


In [101]:
df_copy = df_copy.drop(columns=['Elevator Title'])
df_copy = df_copy.rename(columns={'Port Title': 'Terminal title', 'Address': 'Oblast'})
df_copy

,Terminal title,Crop,Price,Date,Oblast,Director Name,Director Phone,Manager Name,Manager Phone,Terminal type
0,Тіс-Міндобрива ТОВ,Соя(37.5% білок на суху),8450,01.01.2020,NaN,NaN,NaN,NaN,NaN,Port
1,Тіс-Зерно ТОВ,Пшениця 2 клас,5535,01.01.2020,NaN,NaN,NaN,NaN,NaN,Port
2,Тіс-Зерно ТОВ,Пшениця 3 клас,5535,01.01.2020,NaN,NaN,NaN,NaN,NaN,Port
3,Тіс-Зерно ТОВ,Пшениця 4 клас,5420,01.01.2020,NaN,NaN,NaN,NaN,NaN,Port
4,Тіс-Зерно ТОВ,Кукурудза 3 клас,4635,01.01.2020,NaN,NaN,NaN,NaN,NaN,Port
...,...,...,...,...,...,...,...,...,...,...
122973,Радивилівський елеватор ТОВ,Соняшник некласний,21000,17.09.2024,"35500, Рівненська обл., Дубенський р-н, м. Рад...",Порчинський Сергій Іванович,+38 (067) 380-34-33,Мельник Анатолій Іванович,+38 (067) 645-16-82,Elevator
122974,ДПЗКУ ПАТ (філія Новополтавський елеватор),Пшениця 2 клас,8300,17.09.2024,"55645, Миколаївська, обл., Баштанський, р-н",,,,,Elevator
122975,ДПЗКУ ПАТ (філія Новополтавський елеватор),Пшениця 3 клас,8200,17.09.2024,"55645, Миколаївська, обл., Баштанський, р-н",,,,,Elevator
122976,ДПЗКУ ПАТ (філія Новополтавський елеватор),Пшениця 4 клас,7400,17.09.2024,"55645, Миколаївська, обл., Баштанський, р-н",,,,,Elevator


In [102]:
df_copy.loc[df_copy['Terminal title'] == 'Староконстянтинівський елеватор', 'Oblast'] = 'Хмельницька'

In [103]:
def swap_date_format(date_str):
    parts = date_str.split('.')
    if len(parts) == 3:
        return f"{parts[1]}.{parts[0]}.{parts[2]}"
    return date_str

# Apply the function where 'Terminal type' is 'Port'
df_copy['Date'] = df_copy.apply(
    lambda row: swap_date_format(row['Date']) if row['Terminal type'] == 'Port' else row['Date'],
    axis=1
)

# Display the modified DataFrame
df_copy


,Terminal title,Crop,Price,Date,Oblast,Director Name,Director Phone,Manager Name,Manager Phone,Terminal type
0,Тіс-Міндобрива ТОВ,Соя(37.5% білок на суху),8450,01.01.2020,NaN,NaN,NaN,NaN,NaN,Port
1,Тіс-Зерно ТОВ,Пшениця 2 клас,5535,01.01.2020,NaN,NaN,NaN,NaN,NaN,Port
2,Тіс-Зерно ТОВ,Пшениця 3 клас,5535,01.01.2020,NaN,NaN,NaN,NaN,NaN,Port
3,Тіс-Зерно ТОВ,Пшениця 4 клас,5420,01.01.2020,NaN,NaN,NaN,NaN,NaN,Port
4,Тіс-Зерно ТОВ,Кукурудза 3 клас,4635,01.01.2020,NaN,NaN,NaN,NaN,NaN,Port
...,...,...,...,...,...,...,...,...,...,...
122973,Радивилівський елеватор ТОВ,Соняшник некласний,21000,17.09.2024,"35500, Рівненська обл., Дубенський р-н, м. Рад...",Порчинський Сергій Іванович,+38 (067) 380-34-33,Мельник Анатолій Іванович,+38 (067) 645-16-82,Elevator
122974,ДПЗКУ ПАТ (філія Новополтавський елеватор),Пшениця 2 клас,8300,17.09.2024,"55645, Миколаївська, обл., Баштанський, р-н",,,,,Elevator
122975,ДПЗКУ ПАТ (філія Новополтавський елеватор),Пшениця 3 клас,8200,17.09.2024,"55645, Миколаївська, обл., Баштанський, р-н",,,,,Elevator
122976,ДПЗКУ ПАТ (філія Новополтавський елеватор),Пшениця 4 клас,7400,17.09.2024,"55645, Миколаївська, обл., Баштанський, р-н",,,,,Elevator


In [104]:
import os

# Define the file path and sheet name
file_path = r"C:\WORK\CropsandFuelSeptember2020.xlsx"
sheet_name = "Crops(AgroProsperis)"

# Save the DataFrame to the specified Excel file and sheet
df_copy.to_excel(file_path, sheet_name=sheet_name, index=False)

print(f"DataFrame saved to {file_path} in the sheet {sheet_name}.")


DataFrame saved to C:\WORK\CropsandFuelSeptember2020.xlsx in the sheet Crops(AgroProsperis).
